In [1]:
from neo4j import GraphDatabase

# Initialize Neo4j Driver
driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "password"))

# Parse the JSON data and create graph in Neo4j
def create_graph_in_neo4j():
    with open("train.json") as f:
        data = [json.loads(line) for line in f]
    
    with driver.session() as session:
        for record in data:
            paper_id = record["paper"]
            references = record["reference"]
            # Create paper nodes
            session.run("MERGE (p:Paper {id: $paper_id})", paper_id=paper_id)
            # Create reference nodes and directed citation relationships
            for ref in references:
                session.run("MERGE (r:Paper {id: $ref})", ref=ref)
                session.run("MATCH (p:Paper {id: $paper_id}), (r:Paper {id: $ref}) "
                            "MERGE (p)-[:CITES]->(r)",
                            paper_id=paper_id, ref=ref)

create_graph_in_neo4j()


NameError: name 'json' is not defined

In [5]:
import json
from neo4j import GraphDatabase

# Initialize Neo4j Driver
driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "paras2003"))

def create_graph_in_neo4j():
    """
    Parses the JSON data and creates the citation graph in Neo4j.
    Each paper is a node, and each citation is a directed edge.
    """
    with open("train.json") as f:
        data = [json.loads(line) for line in f]
    
    with driver.session() as session:
        for record in data:
            paper_id = record["paper"]
            references = record["reference"]
            # Create paper nodes
            session.run("MERGE (p:Paper {id: $paper_id})", paper_id=paper_id)
            # Create reference nodes and directed citation relationships
            for ref in references:
                session.run("MERGE (r:Paper {id: $ref})", ref=ref)
                session.run("MATCH (p:Paper {id: $paper_id}), (r:Paper {id: $ref}) "
                            "MERGE (p)-[:CITES]->(r)",
                            paper_id=paper_id, ref=ref)

# create_graph_in_neo4j()

from neo4j import GraphDatabase
import csv

def export_graph_from_neo4j(uri="neo4j://localhost:7687", 
                           username="neo4j", 
                           password="paras2003",
                           output_file="graph_edges.csv"):
    """
    Export Neo4j citation graph data into a CSV file without requiring APOC.
    
    Parameters:
    -----------
    uri : str
        Neo4j connection URI
    username : str
        Neo4j username
    password : str
        Neo4j password
    output_file : str
        Path to output CSV file
    """
    driver = GraphDatabase.driver(uri, auth=(username, password))
    
    try:
        with driver.session() as session:
            # Query to get all citations
            query = """
            MATCH (p1:Paper)-[:CITES]->(p2:Paper)
            RETURN p1.id AS source, p2.id AS target
            """
            
            # Execute query and fetch results
            result = session.run(query)
            
            # Write results to CSV
            with open(output_file, 'w', newline='') as f:
                writer = csv.writer(f)
                # Write header
                writer.writerow(['source', 'target'])
                # Write data rows
                for record in result:
                    writer.writerow([record['source'], record['target']])
                    
            print(f"Successfully exported graph data to {output_file}")
            
    except Exception as e:
        print(f"Error exporting graph: {str(e)}")
    finally:
        driver.close()



export_graph_from_neo4j()


Successfully exported graph data to graph_edges.csv


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from itertools import product

# Initialize Spark session
spark = SparkSession.builder.appName("SimRank").getOrCreate()

# Load the graph data from CSV
edges_df = spark.read.csv("graph_edges.csv", header=True)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/15 01:50:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/15 01:50:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
def get_in_neighbors(df, node):
    """ Returns a list of in-neighbors for a given node. """
    return [row['source'] for row in df.filter(df['target'] == node).select('source').collect()]

def simrank(df, u, v, C=0.8, max_iterations=10, tolerance=1e-4):
    """
    Calculate the SimRank similarity between two nodes using Spark DataFrames.
    """
    if u == v:
        return 1.0
    
    sim = {(u, v): 0.0}
    for iteration in range(max_iterations):
        new_sim = sim.copy()
        # Get in-neighbors
        in_neighbors_u = get_in_neighbors(df, u)
        in_neighbors_v = get_in_neighbors(df, v)

        if not in_neighbors_u or not in_neighbors_v:
            return 0.0

        scale = C / (len(in_neighbors_u) * len(in_neighbors_v))
        sim_value = scale * sum(sim.get((w, x), 0) for w, x in product(in_neighbors_u, in_neighbors_v))
        
        new_sim[(u, v)] = sim_value
        
        # Check for convergence
        if abs(new_sim[(u, v)] - sim[(u, v)]) < tolerance:
            break
        
        sim = new_sim
    
    return sim[(u, v)]

# Example query nodes
query_nodes = [2982615777, 1556418098]
decay_factors = [0.7, 0.8, 0.9]

# Run SimRank for each decay factor and query nodes
for C in decay_factors:
    print(f"Decay Factor: {C}")
    for query in query_nodes:
        # Compute similarity for query node against all other nodes in the graph
        similarities = []
        all_nodes = edges_df.select("source").union(edges_df.select("target")).distinct().collect()
        for node in all_nodes:
            similarity = simrank(edges_df, query, node['source'], C=C)
            similarities.append((node['source'], similarity))
        
        # Sort by similarity and print top 10 most similar nodes
        similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
        print(f"Top similarities for node {query}:")
        for node, sim_value in similarities[:10]:
            print(f"Node: {node}, Similarity: {sim_value}")


Decay Factor: 0.7


ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/Users/parasdhiman/opt/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/parasdhiman/opt/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/parasdhiman/opt/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.base/sun.nio.ch.NioSocketImpl.timedAccept(NioSocketImpl.java:708)
	at java.base/sun.nio.ch.NioSocketImpl.accept(NioSocketImpl.java:752)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:675)
	at java.base/java.net.ServerSocket.platformImplAccept(ServerSocket.java:641)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:617)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:574)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:532)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:65)


In [10]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from itertools import product
import pandas as pd
import os
from datetime import datetime
from tqdm.auto import tqdm

def cache_in_neighbors(df):
    """Cache in-neighbors for all nodes to avoid repeated queries."""
    in_neighbors_df = df.groupBy('target').agg(
        F.collect_list('source').alias('in_neighbors')
    ).cache()
    return {row['target']: row['in_neighbors'] for row in in_neighbors_df.collect()}

def batch_simrank(df, query_nodes, target_nodes, C=0.8, max_iterations=10, tolerance=1e-4, 
                  batch_size=1000):
    """
    Calculate SimRank similarities for multiple node pairs in batches.
    """
    # Cache in-neighbors for all nodes
    in_neighbors_dict = cache_in_neighbors(df)
    
    similarities = []
    
    # Process in batches
    for i in range(0, len(target_nodes), batch_size):
        batch_targets = target_nodes[i:i + batch_size]
        batch_results = []
        
        for u in query_nodes:
            for v in batch_targets:
                if u == v:
                    batch_results.append((u, v, 1.0))
                    continue
                
                in_neighbors_u = in_neighbors_dict.get(u, [])
                in_neighbors_v = in_neighbors_dict.get(v, [])
                
                if not in_neighbors_u or not in_neighbors_v:
                    batch_results.append((u, v, 0.0))
                    continue
                
                sim = 0.0
                for _ in range(max_iterations):
                    scale = C / (len(in_neighbors_u) * len(in_neighbors_v))
                    new_sim = scale * sum(
                        sim if w == x else 0.0 
                        for w, x in product(in_neighbors_u, in_neighbors_v)
                    )
                    
                    if abs(new_sim - sim) < tolerance:
                        break
                    sim = new_sim
                
                batch_results.append((u, v, sim))
        
        similarities.extend(batch_results)
    
    return similarities

def compute_and_save_similarities(edges_df, query_nodes, decay_factors, output_dir="simrank_results", 
                                batch_size=1000):
    """
    Compute SimRank similarities in batches and save results.
    """
    os.makedirs(output_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Get all unique nodes once and convert to list
    print("Collecting unique nodes...")
    all_nodes = [row['node'] for row in edges_df.select("source").union(
        edges_df.select("target")).distinct().withColumnRenamed(
        "source", "node").collect()]
    total_nodes = len(all_nodes)
    print(f"Total unique nodes: {total_nodes}")
    
    all_results = []
    top_results = []
    
    # Create progress bars
    decay_pbar = tqdm(decay_factors, desc="Decay Factors", position=0)
    batch_pbar = tqdm(total=total_nodes // batch_size + 1, desc="Batches", position=1)
    
    try:
        for C in decay_pbar:
            decay_pbar.set_description(f"Decay Factor: {C}")
            batch_pbar.reset()
            
            # Process all nodes in batches
            similarities = batch_simrank(
                edges_df, 
                query_nodes, 
                all_nodes, 
                C=C, 
                batch_size=batch_size
            )
            
            # Convert results to DataFrame
            batch_df = pd.DataFrame(similarities, columns=['query_node', 'target_node', 'similarity'])
            batch_df['decay_factor'] = C
            
            # Store all results
            all_results.append(batch_df)
            
            # Get top 10 for each query node
            for query in query_nodes:
                query_results = batch_df[batch_df['query_node'] == query].nlargest(
                    10, 'similarity').copy()
                query_results['rank'] = range(1, 11)
                top_results.append(query_results)
            
            batch_pbar.update(total_nodes // batch_size + 1)
    
    finally:
        batch_pbar.close()
        decay_pbar.close()
    
    # Combine results
    all_results_df = pd.concat(all_results, ignore_index=True)
    top_results_df = pd.concat(top_results, ignore_index=True)
    
    print("\nSaving results to CSV files...")
    
    # Save results with progress bars
    with tqdm(total=3, desc="Saving files") as pbar:
        # Save all results
        all_results_path = os.path.join(output_dir, f'simrank_all_results_{timestamp}.csv')
        all_results_df.to_csv(all_results_path, index=False)
        pbar.update(1)
        
        # Save top results
        top_results_path = os.path.join(output_dir, f'simrank_top_results_{timestamp}.csv')
        top_results_df.to_csv(top_results_path, index=False)
        pbar.update(1)
        
        # Calculate and save summary statistics
        summary_stats = pd.DataFrame({
            'decay_factor': decay_factors,
            'avg_similarity': [all_results_df[all_results_df['decay_factor'] == C]['similarity'].mean() 
                             for C in decay_factors],
            'max_similarity': [all_results_df[all_results_df['decay_factor'] == C]['similarity'].max() 
                             for C in decay_factors],
            'min_similarity': [all_results_df[all_results_df['decay_factor'] == C]['similarity'].min() 
                             for C in decay_factors],
            'std_similarity': [all_results_df[all_results_df['decay_factor'] == C]['similarity'].std() 
                             for C in decay_factors]
        })
        
        summary_path = os.path.join(output_dir, f'simrank_summary_{timestamp}.csv')
        summary_stats.to_csv(summary_path, index=False)
        pbar.update(1)
    
    print(f"\nResults saved in directory: {output_dir}")
    print(f"- All results: {all_results_path}")
    print(f"- Top results: {top_results_path}")
    print(f"- Summary statistics: {summary_path}")
    
    return all_results_df, top_results_df, summary_stats

# Example usage
# if __name__ == "__main__":
# Initialize Spark session
spark = SparkSession.builder.appName("SimRank").getOrCreate()

# Load the graph data from CSV
edges_df = spark.read.csv("graph_edges.csv", header=True)

# Example query nodes and decay factors
query_nodes = [2982615777, 1556418098]
decay_factors = [0.7, 0.8, 0.9]

# Compute similarities and save results
all_results, top_results, summary = compute_and_save_similarities(
    edges_df,
    query_nodes=query_nodes,
    decay_factors=decay_factors,
    output_dir="simrank_results"
)

Total unique nodes: 359143


Decay Factors:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/360 [00:00<?, ?it/s]

24/11/15 02:00:26 WARN CacheManager: Asked to cache already cached data.        
24/11/15 02:00:29 WARN CacheManager: Asked to cache already cached data.



Saving results to CSV files...


Saving files:   0%|          | 0/3 [00:00<?, ?it/s]


Results saved in directory: simrank_results
- All results: simrank_results/simrank_all_results_20241115_020021.csv
- Top results: simrank_results/simrank_top_results_20241115_020021.csv
- Summary statistics: simrank_results/simrank_summary_20241115_020021.csv


In [31]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
import os
from datetime import datetime
from tqdm.auto import tqdm

def cache_in_neighbors(df):
    """Cache in-neighbors for all nodes to avoid repeated queries."""
    in_neighbors_df = df.groupBy('target').agg(
        F.collect_list('source').alias('in_neighbors')
    ).cache()
    return {row['target']: row['in_neighbors'] for row in in_neighbors_df.collect()}

def compute_simrank_similarity(a, b, in_neighbors_dict, C, max_iterations=10, tolerance=1e-4):
    """Compute SimRank similarity between two nodes."""
    if a == b:
        return 1.0
        
    in_neighbors_a = in_neighbors_dict.get(a, [])
    in_neighbors_b = in_neighbors_dict.get(b, [])
    
    if not in_neighbors_a or not in_neighbors_b:
        return 0.0
    
    # Initialize similarity matrix for in-neighbors
    sim_matrix = {}
    for na in in_neighbors_a:
        for nb in in_neighbors_b:
            if na == nb:
                sim_matrix[(na, nb)] = 1.0
            else:
                sim_matrix[(na, nb)] = 0.0
    
    # Iterate until convergence
    for _ in range(max_iterations):
        new_sim_matrix = {}
        max_diff = 0.0
        
        for na in in_neighbors_a:
            for nb in in_neighbors_b:
                if na == nb:
                    new_sim_matrix[(na, nb)] = 1.0
                    continue
                    
                in_na = in_neighbors_dict.get(na, [])
                in_nb = in_neighbors_dict.get(nb, [])
                
                if not in_na or not in_nb:
                    new_sim_matrix[(na, nb)] = 0.0
                    continue
                
                sum_sim = 0.0
                for i in in_na:
                    for j in in_nb:
                        sum_sim += sim_matrix.get((i, j), 0.0)
                
                new_sim = (C / (len(in_na) * len(in_nb))) * sum_sim
                new_sim_matrix[(na, nb)] = new_sim
                max_diff = max(max_diff, abs(new_sim - sim_matrix.get((na, nb), 0.0)))
        
        sim_matrix = new_sim_matrix
        if max_diff < tolerance:
            break
    
    # Calculate final similarity
    sum_sim = 0.0
    for na in in_neighbors_a:
        for nb in in_neighbors_b:
            sum_sim += sim_matrix.get((na, nb), 0.0)
    
    return (C / (len(in_neighbors_a) * len(in_neighbors_b))) * sum_sim

def compute_simrank(df, query_nodes, C=0.8, max_iterations=10, tolerance=1e-4):
    """Compute SimRank similarities for given query nodes."""
    print("Caching in-neighbors...")
    in_neighbors_dict = cache_in_neighbors(df)
    
    # Get all unique nodes
    all_nodes = set([row['node'] for row in df.select("source").union(
        df.select("target")).distinct().withColumnRenamed("source", "node").collect()])
    print(f"Total unique nodes: {len(all_nodes)}")
    
    results = []
    for query_node in tqdm(query_nodes, desc="Processing query nodes"):
        node_results = []
        for target_node in tqdm(all_nodes, desc=f"Computing similarities for node {query_node}", leave=False):
            sim = compute_simrank_similarity(
                query_node, 
                target_node, 
                in_neighbors_dict, 
                C,
                max_iterations,
                tolerance
            )
            node_results.append((query_node, target_node, sim))
        results.extend(node_results)
    
    return pd.DataFrame(results, columns=['query_node', 'target_node', 'similarity'])

def run_simrank_analysis(edges_df, query_nodes, decay_factors, output_dir="simrank_results"):
    """Run SimRank analysis for multiple decay factors and save results."""
    os.makedirs(output_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    all_results = []
    
    for C in tqdm(decay_factors, desc="Processing decay factors"):
        print(f"\nComputing SimRank with decay factor C = {C}")
        
        results_df = compute_simrank(edges_df, query_nodes, C=C)
        results_df['decay_factor'] = C
        all_results.append(results_df)
        
        # Save intermediate results
        results_df.to_csv(
            f"{output_dir}/simrank_results_C{C}_{timestamp}.csv",
            index=False
        )
    
    final_results = pd.concat(all_results, ignore_index=True)
    
    # Save complete results
    final_results.to_csv(
        f"{output_dir}/simrank_all_results_{timestamp}.csv",
        index=False
    )
    
    # Generate and save top 10 results
    top_results = []
    for C in decay_factors:
        for query in query_nodes:
            top_10 = final_results[
                (final_results['decay_factor'] == C) & 
                (final_results['query_node'] == query)
            ].nlargest(10, 'similarity')
            top_10['rank'] = range(1, 11)
            top_results.append(top_10)
    
    top_results_df = pd.concat(top_results, ignore_index=True)
    top_results_df.to_csv(
        f"{output_dir}/simrank_top_results_{timestamp}.csv",
        index=False
    )
    
    # Print summary
    print("\nTop 5 most similar nodes for each query node and decay factor:")
    for C in decay_factors:
        print(f"\nDecay factor C = {C}")
        for query in query_nodes:
            print(f"\nQuery node: {query}")
            top_5 = top_results_df[
                (top_results_df['decay_factor'] == C) & 
                (top_results_df['query_node'] == query)
            ].head()
            print(top_5[['target_node', 'similarity', 'rank']].to_string())
    
    return final_results, top_results_df

# Run the analysis
query_nodes = [2982615777, 1556418098]
decay_factors = [0.7, 0.8, 0.9]

final_results, top_results = run_simrank_analysis(
    edges_df,
    query_nodes=query_nodes,
    decay_factors=decay_factors,
    output_dir="simrank_results"
)

Processing decay factors:   0%|          | 0/3 [00:00<?, ?it/s]


Computing SimRank with decay factor C = 0.7
Caching in-neighbors...


RuntimeError: SparkContext or SparkSession should be created first.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
import os
from datetime import datetime
from tqdm.auto import tqdm

class SimRankAnalyzer:
    def __init__(self, spark_session=None):
        """Initialize SimRank analyzer with optional Spark session."""
        self.spark = spark_session or SparkSession.builder \
            .appName("SimRank Analysis") \
            .getOrCreate()
        
    def create_graph(self, edges_data):
        """Create graph from citation data."""
        # Convert edges data to Spark DataFrame
        edges_df = self.spark.createDataFrame(edges_data, ["source", "target"])
        
        # Cache the DataFrame as we'll be using it repeatedly
        return edges_df.cache()
    
    def cache_in_neighbors(self, df):
        """Cache in-neighbors for all nodes to avoid repeated queries."""
        in_neighbors_df = df.groupBy('target').agg(
            F.collect_list('source').alias('in_neighbors')
        ).cache()
        return {row['target']: row['in_neighbors'] for row in in_neighbors_df.collect()}
    
    def compute_simrank_similarity(self, a, b, in_neighbors_dict, C, max_iterations=10, tolerance=1e-4):
        """Compute SimRank similarity between two nodes."""
        if a == b:
            return 1.0
            
        in_neighbors_a = in_neighbors_dict.get(a, [])
        in_neighbors_b = in_neighbors_dict.get(b, [])
        
        if not in_neighbors_a or not in_neighbors_b:
            return 0.0
        
        # Initialize similarity matrix for in-neighbors
        sim_matrix = {(na, nb): 1.0 if na == nb else 0.0 
                     for na in in_neighbors_a 
                     for nb in in_neighbors_b}
        
        # Iterate until convergence
        for _ in range(max_iterations):
            new_sim_matrix = {}
            max_diff = 0.0
            
            for na in in_neighbors_a:
                for nb in in_neighbors_b:
                    if na == nb:
                        new_sim_matrix[(na, nb)] = 1.0
                        continue
                        
                    in_na = in_neighbors_dict.get(na, [])
                    in_nb = in_neighbors_dict.get(nb, [])
                    
                    if not in_na or not in_nb:
                        new_sim_matrix[(na, nb)] = 0.0
                        continue
                    
                    sum_sim = sum(sim_matrix.get((i, j), 0.0) 
                                for i in in_na 
                                for j in in_nb)
                    
                    new_sim = (C / (len(in_na) * len(in_nb))) * sum_sim
                    new_sim_matrix[(na, nb)] = new_sim
                    max_diff = max(max_diff, abs(new_sim - sim_matrix.get((na, nb), 0.0)))
            
            sim_matrix = new_sim_matrix
            if max_diff < tolerance:
                break
        
        # Calculate final similarity
        sum_sim = sum(sim_matrix.get((na, nb), 0.0) 
                     for na in in_neighbors_a 
                     for nb in in_neighbors_b)
        
        return (C / (len(in_neighbors_a) * len(in_neighbors_b))) * sum_sim
    
    def compute_simrank(self, df, query_nodes, C=0.8, max_iterations=10, tolerance=1e-4):
        """Compute SimRank similarities for given query nodes."""
        print("Caching in-neighbors...")
        in_neighbors_dict = self.cache_in_neighbors(df)
        
        # Get all unique nodes
        all_nodes = set(row['node'] for row in df.select("source")
                       .union(df.select("target"))
                       .distinct()
                       .withColumnRenamed("source", "node")
                       .collect())
        print(f"Total unique nodes: {len(all_nodes)}")
        
        results = []
        for query_node in tqdm(query_nodes, desc="Processing query nodes"):
            node_results = []
            for target_node in tqdm(all_nodes, desc=f"Computing similarities for node {query_node}", leave=False):
                sim = self.compute_simrank_similarity(
                    query_node, 
                    target_node, 
                    in_neighbors_dict, 
                    C,
                    max_iterations,
                    tolerance
                )
                node_results.append((query_node, target_node, sim))
            results.extend(node_results)
        
        return pd.DataFrame(results, columns=['query_node', 'target_node', 'similarity'])
    
    def analyze_and_save_results(self, edges_df, query_nodes, decay_factors, output_dir="simrank_results"):
        """Run SimRank analysis with multiple decay factors and save results."""
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        all_results = []
        
        for C in decay_factors:
            print(f"\nComputing SimRank with decay factor C = {C}")
            results_df = self.compute_simrank(edges_df, query_nodes, C=C)
            results_df['decay_factor'] = C
            all_results.append(results_df)
            
            # Save intermediate results
            output_path = f"{output_dir}/simrank_results_C{C}_{timestamp}.csv"
            results_df.to_csv(output_path, index=False)
            print(f"Saved intermediate results to {output_path}")
        
        # Combine all results
        final_results = pd.concat(all_results, ignore_index=True)
        
        # Save complete results
        final_path = f"{output_dir}/simrank_all_results_{timestamp}.csv"
        final_results.to_csv(final_path, index=False)
        print(f"Saved complete results to {final_path}")
        
        # Generate and save top 10 results
        top_results = []
        for C in decay_factors:
            for query in query_nodes:
                mask = (final_results['decay_factor'] == C) & (final_results['query_node'] == query)
                top_10 = final_results[mask].nlargest(10, 'similarity')
                top_10['rank'] = range(1, 11)
                top_results.append(top_10)
        
        top_results_df = pd.concat(top_results, ignore_index=True)
        top_path = f"{output_dir}/simrank_top_results_{timestamp}.csv"
        top_results_df.to_csv(top_path, index=False)
        print(f"Saved top results to {top_path}")
        
        self._print_summary(top_results_df, decay_factors, query_nodes)
        
        return final_results, top_results_df
    
    def _print_summary(self, top_results_df, decay_factors, query_nodes):
        """Print summary of top results."""
        print("\nTop 5 most similar nodes for each query node and decay factor:")
        for C in decay_factors:
            print(f"\nDecay factor C = {C}")
            for query in query_nodes:
                print(f"\nQuery node: {query}")
                mask = (top_results_df['decay_factor'] == C) & (top_results_df['query_node'] == query)
                top_5 = top_results_df[mask].head()
                print(top_5[['target_node', 'similarity', 'rank']].to_string())

# Example usage
def main():
    # Initialize analyzer
    analyzer = SimRankAnalyzer()
    
    # Create example edges DataFrame (replace with your actual data)
    
    edges_df = analyzer.create_graph(edges_data)
    
    # Set parameters
    query_nodes = [2982615777, 1556418098]
    decay_factors = [0.7, 0.8, 0.9]
    
    # Run analysis
    final_results, top_results = analyzer.analyze_and_save_results(
        edges_df,
        query_nodes=query_nodes,
        decay_factors=decay_factors,
        output_dir="simrank_results"
    )
    
    return final_results, top_results

if __name__ == "__main__":
    main()

24/11/15 02:07:28 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.



Computing SimRank with decay factor C = 0.7
Caching in-neighbors...


Total unique nodes: 3


Processing query nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Computing similarities for node 2982615777:   0%|          | 0/3 [00:00<?, ?it/s]

...

Processing query nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Computing similarities for node 2982615777:   0%|          | 0/3 [00:00<?, ?it/s]

Computing similarities for node 1556418098:   0%|          | 0/3 [00:00<?, ?it/s]

Saved intermediate results to simrank_results/simrank_results_C0.9_20241115_020728.csv
Saved complete results to simrank_results/simrank_all_results_20241115_020728.csv


ValueError: Length of values (10) does not match length of index (3)

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
import os
from datetime import datetime
from tqdm.auto import tqdm
from collections import defaultdict

class SimRankAnalyzer:
    def __init__(self, spark_session=None):
        """Initialize SimRank analyzer with optional Spark session."""
        self.spark = spark_session or SparkSession.builder \
            .appName("SimRank Analysis") \
            .getOrCreate()
        
    def create_graph(self, edges_data):
        """Create graph from citation data."""
        edges_df = self.spark.createDataFrame(edges_data, ["source", "target"])
        return edges_df.cache()
    
    def cache_in_neighbors(self, df):
        """Cache in-neighbors for all nodes to avoid repeated queries."""
        in_neighbors_df = df.groupBy('target').agg(
            F.collect_list('source').alias('in_neighbors')
        ).cache()
        return {row['target']: row['in_neighbors'] for row in in_neighbors_df.collect()}
    
    def compute_simrank_similarity(self, a, b, in_neighbors_dict, C, max_iterations=10, tolerance=1e-4):
        """Compute SimRank similarity between two nodes."""
        if a == b:
            return 1.0
            
        in_neighbors_a = in_neighbors_dict.get(a, [])
        in_neighbors_b = in_neighbors_dict.get(b, [])
        
        if not in_neighbors_a or not in_neighbors_b:
            return 0.0
        
        # Initialize similarity scores for all node pairs that might be needed
        all_nodes = set(in_neighbors_a + in_neighbors_b)
        sim_scores = defaultdict(lambda: defaultdict(float))
        for node in all_nodes:
            sim_scores[node][node] = 1.0
        
        # SimRank iterations
        for _ in range(max_iterations):
            new_scores = defaultdict(lambda: defaultdict(float))
            max_diff = 0.0
            
            # Update similarity for each pair of nodes
            for na in in_neighbors_a:
                for nb in in_neighbors_b:
                    if na == nb:
                        new_scores[na][nb] = 1.0
                        continue
                    
                    # Get in-neighbors of the current pair
                    in_na = in_neighbors_dict.get(na, [])
                    in_nb = in_neighbors_dict.get(nb, [])
                    
                    if not in_na or not in_nb:
                        continue
                    
                    # Compute similarity using the SimRank formula
                    similarity_sum = sum(sim_scores[i][j] 
                                      for i in in_na 
                                      for j in in_nb)
                    
                    new_sim = (C / (len(in_na) * len(in_nb))) * similarity_sum
                    new_scores[na][nb] = new_sim
                    new_scores[nb][na] = new_sim  # SimRank is symmetric
                    
                    # Track maximum change for convergence check
                    old_sim = sim_scores[na][nb]
                    max_diff = max(max_diff, abs(new_sim - old_sim))
            
            # Update similarity scores
            sim_scores = new_scores
            
            # Check for convergence
            if max_diff < tolerance:
                break
        
        # Calculate final similarity between input nodes
        similarity_sum = sum(sim_scores[i][j] 
                           for i in in_neighbors_a 
                           for j in in_neighbors_b)
        
        if not in_neighbors_a or not in_neighbors_b:
            return 0.0
            
        return (C / (len(in_neighbors_a) * len(in_neighbors_b))) * similarity_sum
    
    def compute_simrank(self, df, query_nodes, C=0.8, max_iterations=10, tolerance=1e-4):
        """Compute SimRank similarities for given query nodes."""
        print("Caching in-neighbors...")
        in_neighbors_dict = self.cache_in_neighbors(df)
        
        # Get all unique nodes from the graph
        all_nodes = set(row['node'] for row in df.select("source")
                       .union(df.select("target"))
                       .distinct()
                       .withColumnRenamed("source", "node")
                       .collect())
        print(f"Total unique nodes: {len(all_nodes)}")
        
        results = []
        for query_node in tqdm(query_nodes, desc="Processing query nodes"):
            node_results = []
            for target_node in tqdm(all_nodes, desc=f"Computing similarities for node {query_node}", leave=False):
                sim = self.compute_simrank_similarity(
                    query_node, 
                    target_node, 
                    in_neighbors_dict, 
                    C,
                    max_iterations,
                    tolerance
                )
                node_results.append((query_node, target_node, sim))
            results.extend(node_results)
        
        return pd.DataFrame(results, columns=['query_node', 'target_node', 'similarity'])
    
    def analyze_and_save_results(self, edges_df, query_nodes, decay_factors, output_dir="simrank_results"):
        """Run SimRank analysis with multiple decay factors and save results."""
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        all_results = []
        
        for C in decay_factors:
            print(f"\nComputing SimRank with decay factor C = {C}")
            results_df = self.compute_simrank(edges_df, query_nodes, C=C)
            results_df['decay_factor'] = C
            all_results.append(results_df)
            
            output_path = f"{output_dir}/simrank_results_C{C}_{timestamp}.csv"
            results_df.to_csv(output_path, index=False)
            print(f"Saved intermediate results to {output_path}")
        
        final_results = pd.concat(all_results, ignore_index=True)
        
        final_path = f"{output_dir}/simrank_all_results_{timestamp}.csv"
        final_results.to_csv(final_path, index=False)
        print(f"Saved complete results to {final_path}")
        
        # Generate top results
        top_results = []
        for C in decay_factors:
            for query in query_nodes:
                mask = (final_results['decay_factor'] == C) & (final_results['query_node'] == query)
                subset = final_results[mask].nlargest(10, 'similarity')
                subset = subset.copy()
                subset['rank'] = range(1, len(subset) + 1)
                top_results.append(subset)
        
        top_results_df = pd.concat(top_results, ignore_index=True)
        top_path = f"{output_dir}/simrank_top_results_{timestamp}.csv"
        top_results_df.to_csv(top_path, index=False)
        print(f"Saved top results to {top_path}")
        
        self._print_summary(top_results_df, decay_factors, query_nodes)
        
        return final_results, top_results_df
    
    def _print_summary(self, top_results_df, decay_factors, query_nodes):
        """Print summary of top results."""
        print("\nTop 5 most similar nodes for each query node and decay factor:")
        for C in decay_factors:
            print(f"\nDecay factor C = {C}")
            for query in query_nodes:
                print(f"\nQuery node: {query}")
                mask = (top_results_df['decay_factor'] == C) & (top_results_df['query_node'] == query)
                top_5 = top_results_df[mask].head()
                if not top_5.empty:
                    print(top_5[['target_node', 'similarity', 'rank']].to_string())
                else:
                    print("No results found")


analyzer = SimRankAnalyzer()

# Create example edges DataFrame
edges_data = [
    (2982615777, 2087551257),
    (2982615777, 2044328306),
    (2044328306, 2087551257),  # Added more edges to show similarity
    (2087551257, 2044328306),  # Added circular reference
]

edges_df = analyzer.create_graph(edges_data)

# Set parameters
query_nodes = [2982615777, 1556418098]
decay_factors = [0.7, 0.8, 0.9]

# Run analysis
final_results, top_results = analyzer.analyze_and_save_results(
    edges_df,
    query_nodes=query_nodes,
    decay_factors=decay_factors,
    output_dir="simrank_results"
)

print(final_results, top_results)




Computing SimRank with decay factor C = 0.7
Caching in-neighbors...
Total unique nodes: 3


Processing query nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Computing similarities for node 2982615777:   0%|          | 0/3 [00:00<?, ?it/s]

Computing similarities for node 1556418098:   0%|          | 0/3 [00:00<?, ?it/s]

Saved intermediate results to simrank_results/simrank_results_C0.7_20241115_021205.csv

Computing SimRank with decay factor C = 0.8
Caching in-neighbors...
Total unique nodes: 3


...

24/11/15 02:12:06 WARN CacheManager: Asked to cache already cached data.


Processing query nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Computing similarities for node 2982615777:   0%|          | 0/3 [00:00<?, ?it/s]

Computing similarities for node 1556418098:   0%|          | 0/3 [00:00<?, ?it/s]

Saved intermediate results to simrank_results/simrank_results_C0.9_20241115_021205.csv
Saved complete results to simrank_results/simrank_all_results_20241115_021205.csv
Saved top results to simrank_results/simrank_top_results_20241115_021205.csv

Top 5 most similar nodes for each query node and decay factor:

Decay factor C = 0.7

Query node: 2982615777
   target_node  similarity  rank
0   2982615777         1.0     1
1   2044328306         0.0     2
2   2087551257         0.0     3

Query node: 1556418098
   target_node  similarity  rank
3   2982615777         0.0     1
4   2044328306         0.0     2
5   2087551257         0.0     3

Decay factor C = 0.8

Query node: 2982615777
   target_node  similarity  rank
6   2982615777         1.0     1
7   2044328306         0.0     2
8   2087551257         0.0     3

Query node: 1556418098
    target_node  similarity  rank
9    2982615777         0.0     1
10   2044328306         0.0     2
11   2087551257         0.0     3

Decay factor C = 

In [16]:
from pyspark.sql import SparkSession
from neo4j import GraphDatabase
import pandas as pd
import os
from datetime import datetime
from tqdm.auto import tqdm
from collections import defaultdict

class CitationGraphAnalyzer:
    def __init__(self, neo4j_uri="bolt://localhost:7687", 
                 neo4j_user="neo4j", neo4j_password="paras2003"):
        """Initialize with Neo4j and Spark connections"""
        # Initialize Neo4j connection
        self.driver = GraphDatabase.driver(neo4j_uri, 
                                         auth=(neo4j_user, neo4j_password))
        
        # Initialize Spark session
        self.spark = SparkSession.builder \
            .appName("Citation Graph Analysis") \
            .config("spark.driver.memory", "4g") \
            .config("spark.executor.memory", "4g") \
            .getOrCreate()
    
    def create_neo4j_graph(self, papers_data):
        """Create graph in Neo4j from citation data"""
        with self.driver.session() as session:
            # Clear existing data
            session.run("MATCH (n) DETACH DELETE n")
            
            # Create paper nodes
            for paper_id in papers_data.keys():
                session.run("""
                    CREATE (p:Paper {id: $paper_id})
                """, paper_id=paper_id)
            
            # Create citation relationships
            for citing_paper, cited_papers in papers_data.items():
                if cited_papers:  # Only create edges if there are references
                    for cited_paper in cited_papers:
                        session.run("""
                            MATCH (citing:Paper {id: $citing_id})
                            MATCH (cited:Paper {id: $cited_id})
                            CREATE (citing)-[:CITES]->(cited)
                        """, citing_id=citing_paper, cited_id=cited_paper)
                        
    def get_graph_data(self):
        """Extract graph data from Neo4j for Spark processing"""
        with self.driver.session() as session:
            # Get all citation relationships
            result = session.run("""
                MATCH (p1:Paper)-[:CITES]->(p2:Paper)
                RETURN p1.id as source, p2.id as target
            """)
            edges = [(record["source"], record["target"]) for record in result]
            
            return edges
    
    def compute_simrank_similarity(self, a, b, in_neighbors_dict, C, max_iterations=10, tolerance=1e-4):
        """Compute SimRank similarity between two nodes"""
        if a == b:
            return 1.0
            
        in_neighbors_a = in_neighbors_dict.get(a, [])
        in_neighbors_b = in_neighbors_dict.get(b, [])
        
        if not in_neighbors_a or not in_neighbors_b:
            return 0.0
        
        # Initialize similarity scores
        sim_scores = defaultdict(lambda: defaultdict(float))
        for node in set(in_neighbors_a + in_neighbors_b):
            sim_scores[node][node] = 1.0
        
        # SimRank iterations
        for _ in range(max_iterations):
            new_scores = defaultdict(lambda: defaultdict(float))
            max_diff = 0.0
            
            for na in in_neighbors_a:
                for nb in in_neighbors_b:
                    if na == nb:
                        new_scores[na][nb] = 1.0
                        continue
                    
                    in_na = in_neighbors_dict.get(na, [])
                    in_nb = in_neighbors_dict.get(nb, [])
                    
                    if not in_na or not in_nb:
                        continue
                    
                    similarity_sum = sum(sim_scores[i][j] 
                                      for i in in_na 
                                      for j in in_nb)
                    
                    new_sim = (C / (len(in_na) * len(in_nb))) * similarity_sum
                    new_scores[na][nb] = new_sim
                    new_scores[nb][na] = new_sim
                    
                    max_diff = max(max_diff, abs(new_sim - sim_scores[na][nb]))
            
            sim_scores = new_scores
            
            if max_diff < tolerance:
                break
        
        similarity_sum = sum(sim_scores[i][j] 
                           for i in in_neighbors_a 
                           for j in in_neighbors_b)
        
        if not in_neighbors_a or not in_neighbors_b:
            return 0.0
            
        return (C / (len(in_neighbors_a) * len(in_neighbors_b))) * similarity_sum
    
    def analyze_citation_graph(self, query_nodes, decay_factors, output_dir="simrank_results"):
        """Run SimRank analysis on citation graph"""
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Get graph data from Neo4j
        edges = self.get_graph_data()
        edges_df = self.spark.createDataFrame(edges, ["source", "target"])
        
        # Cache in-neighbors
        in_neighbors_dict = self._cache_in_neighbors(edges_df)
        
        all_results = []
        
        for C in decay_factors:
            print(f"\nComputing SimRank with decay factor C = {C}")
            
            results = []
            # Get all unique nodes
            all_nodes = set([row['node'] for row in edges_df.select("source")
                           .union(edges_df.select("target"))
                           .distinct()
                           .withColumnRenamed("source", "node")
                           .collect()])
            
            for query_node in tqdm(query_nodes, desc="Processing query nodes"):
                node_results = []
                for target_node in tqdm(all_nodes, desc=f"Computing similarities for node {query_node}", leave=False):
                    sim = self.compute_simrank_similarity(
                        query_node,
                        target_node,
                        in_neighbors_dict,
                        C
                    )
                    node_results.append((query_node, target_node, sim))
                results.extend(node_results)
            
            results_df = pd.DataFrame(results, columns=['query_node', 'target_node', 'similarity'])
            results_df['decay_factor'] = C
            all_results.append(results_df)
            
            # Save intermediate results
            output_path = f"{output_dir}/simrank_results_C{C}_{timestamp}.csv"
            results_df.to_csv(output_path, index=False)
            
        return self._save_and_summarize_results(all_results, query_nodes, decay_factors, timestamp, output_dir)
    
    def _cache_in_neighbors(self, edges_df):
        """Cache in-neighbors for all nodes"""
        in_neighbors = edges_df.groupBy('target').agg(
            F.collect_list('source').alias('in_neighbors')
        ).collect()
        return {row['target']: row['in_neighbors'] for row in in_neighbors}
    
    def _save_and_summarize_results(self, all_results, query_nodes, decay_factors, timestamp, output_dir):
        """Save and summarize final results"""
        final_results = pd.concat(all_results, ignore_index=True)
        
        # Save complete results
        final_path = f"{output_dir}/simrank_all_results_{timestamp}.csv"
        final_results.to_csv(final_path, index=False)
        
        # Generate top results
        top_results = []
        for C in decay_factors:
            for query in query_nodes:
                mask = (final_results['decay_factor'] == C) & (final_results['query_node'] == query)
                subset = final_results[mask].nlargest(10, 'similarity')
                subset = subset.copy()
                subset['rank'] = range(1, len(subset) + 1)
                top_results.append(subset)
        
        top_results_df = pd.concat(top_results, ignore_index=True)
        top_path = f"{output_dir}/simrank_top_results_{timestamp}.csv"
        top_results_df.to_csv(top_path, index=False)
        
        self._print_summary(top_results_df, decay_factors, query_nodes)
        
        return final_results, top_results_df
    
    def _print_summary(self, top_results_df, decay_factors, query_nodes):
        """Print summary of results"""
        print("\nTop 5 most similar nodes for each query node and decay factor:")
        for C in decay_factors:
            print(f"\nDecay factor C = {C}")
            for query in query_nodes:
                print(f"\nQuery node: {query}")
                mask = (top_results_df['decay_factor'] == C) & (top_results_df['query_node'] == query)
                top_5 = top_results_df[mask].head()
                if not top_5.empty:
                    print(top_5[['target_node', 'similarity', 'rank']].to_string())
                else:
                    print("No results found")
    
    def close(self):
        """Close Neo4j and Spark connections"""
        self.driver.close()
        self.spark.stop()

# Example usage
# if __name__ == "__main__":
    # Sample citation data
papers_data = {
    2982615777: [2087551257, 2044328306],
    2044328306: [2087551257],
    2087551257: [2044328306],
    1556418098: []  # Paper with no references
}

# Initialize analyzer
analyzer = CitationGraphAnalyzer()

try:
    # Create Neo4j graph
    print("Creating Neo4j graph...")
    analyzer.create_neo4j_graph(papers_data)
    
    # Run analysis
    query_nodes = [2982615777, 1556418098]
    decay_factors = [0.7, 0.8, 0.9]
    
    print("Running SimRank analysis...")
    final_results, top_results = analyzer.analyze_citation_graph(
        query_nodes=query_nodes,
        decay_factors=decay_factors
    )
    
finally:
    # Clean up connections
    analyzer.close()

24/11/15 02:16:00 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Creating Neo4j graph...
Running SimRank analysis...

Computing SimRank with decay factor C = 0.7


Processing query nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Computing similarities for node 2982615777:   0%|          | 0/3 [00:00<?, ?it/s]

Computing similarities for node 1556418098:   0%|          | 0/3 [00:00<?, ?it/s]

...


Computing SimRank with decay factor C = 0.9


Processing query nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Computing similarities for node 2982615777:   0%|          | 0/3 [00:00<?, ?it/s]

Computing similarities for node 1556418098:   0%|          | 0/3 [00:00<?, ?it/s]


Top 5 most similar nodes for each query node and decay factor:

Decay factor C = 0.7

Query node: 2982615777
   target_node  similarity  rank
0   2982615777         1.0     1
1   2044328306         0.0     2
2   2087551257         0.0     3

Query node: 1556418098
   target_node  similarity  rank
3   2982615777         0.0     1
4   2044328306         0.0     2
5   2087551257         0.0     3

Decay factor C = 0.8

Query node: 2982615777
   target_node  similarity  rank
6   2982615777         1.0     1
7   2044328306         0.0     2
8   2087551257         0.0     3

Query node: 1556418098
    target_node  similarity  rank
9    2982615777         0.0     1
10   2044328306         0.0     2
11   2087551257         0.0     3

Decay factor C = 0.9

Query node: 2982615777
    target_node  similarity  rank
12   2982615777         1.0     1
13   2044328306         0.0     2
14   2087551257         0.0     3

Query node: 1556418098
    target_node  similarity  rank
15   2982615777         

24/11/15 02:26:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Creating Neo4j graph...
Running SimRank analysis...



Computing SimRank with decay factor C = 0.7


Processing query nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Computing similarities for node 2982615777:   0%|          | 0/3 [00:00<?, ?it/s]

TypeError: compute_simrank_similarity() takes 4 positional arguments but 5 were given

In [ ]:
from pyspark.sql import SparkSession
from neo4j import GraphDatabase
import pandas as pd
import os
from datetime import datetime
from tqdm.auto import tqdm
from collections import defaultdict

class CitationGraphAnalyzer:
    def __init__(self, neo4j_uri="bolt://localhost:7687", 
                 neo4j_user="neo4j", neo4j_password="paras2003"):
        """Initialize with Neo4j and Spark connections"""
        # Initialize Neo4j connection
        self.driver = GraphDatabase.driver(neo4j_uri, 
                                         auth=(neo4j_user, neo4j_password))
        
        # Initialize Spark session
        self.spark = SparkSession.builder \
            .appName("Citation Graph Analysis") \
            .config("spark.driver.memory", "4g") \
            .config("spark.executor.memory", "4g") \
            .getOrCreate()
    
    def create_neo4j_graph(self, papers_data):
        """Create graph in Neo4j from citation data"""
        with self.driver.session() as session:
            # Clear existing data
            session.run("MATCH (n) DETACH DELETE n")
            
            # Create paper nodes
            for paper_id in papers_data.keys():
                session.run("""
                    CREATE (p:Paper {id: $paper_id})
                """, paper_id=paper_id)
            
            # Create citation relationships
            for citing_paper, cited_papers in papers_data.items():
                if cited_papers:  # Only create edges if there are references
                    for cited_paper in cited_papers:
                        session.run("""
                            MATCH (citing:Paper {id: $citing_id})
                            MATCH (cited:Paper {id: $cited_id})
                            CREATE (citing)-[:CITES]->(cited)
                        """, citing_id=citing_paper, cited_id=cited_paper)
                        
    def get_graph_data(self):
        """Extract graph data from Neo4j for Spark processing"""
        with self.driver.session() as session:
            # Get all citation relationships
            result = session.run("""
                MATCH (p1:Paper)-[:CITES]->(p2:Paper)
                RETURN p1.id as source, p2.id as target
            """)
            edges = [(record["source"], record["target"]) for record in result]
            
            return edges
    
    from pyspark.sql import functions as F

    def compute_simrank_similarity(self, a, b, in_neighbors_dict, C):
        """Compute SimRank similarity between two nodes"""
        if a == b:
            return 1.0  # self-similarity is always 1.0
        
        in_neighbors_a = in_neighbors_dict.get(a, [])
        in_neighbors_b = in_neighbors_dict.get(b, [])
        
        if not in_neighbors_a or not in_neighbors_b:
            return 0.0
        
        # Initialize similarity scores
        sim_scores = defaultdict(lambda: defaultdict(float))
        for node in set(in_neighbors_a + in_neighbors_b):
            sim_scores[node][node] = 1.0
        
        # SimRank iterations (you can adjust max_iterations/tolerance as needed)
        max_iterations = 10
        tolerance = 1e-4
        for _ in range(max_iterations):
            new_scores = defaultdict(lambda: defaultdict(float))
            max_diff = 0.0
            
            for na in in_neighbors_a:
                for nb in in_neighbors_b:
                    if na == nb:
                        new_scores[na][nb] = 1.0
                        continue
                    
                    in_na = in_neighbors_dict.get(na, [])
                    in_nb = in_neighbors_dict.get(nb, [])
                    
                    if not in_na or not in_nb:
                        continue
                    
                    similarity_sum = sum(sim_scores[i][j] 
                                        for i in in_na 
                                        for j in in_nb)
                    
                    new_sim = (C / (len(in_na) * len(in_nb))) * similarity_sum
                    new_scores[na][nb] = new_sim
                    new_scores[nb][na] = new_sim
                    
                    max_diff = max(max_diff, abs(new_sim - sim_scores[na][nb]))
            
            sim_scores = new_scores
            
            if max_diff < tolerance:
                break
        
        similarity_sum = sum(sim_scores[i][j] 
                            for i in in_neighbors_a 
                            for j in in_neighbors_b)
        
        return (C / (len(in_neighbors_a) * len(in_neighbors_b))) * similarity_sum


    def analyze_citation_graph(self, query_nodes, decay_factors, output_dir="simrank_results"):
        """Run SimRank analysis on citation graph"""
        os.makedirs(output_dir, exist_ok=True)
        assert os.path.isdir(output_dir), f"Could not create or access directory: {output_dir}"

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Get graph data from Neo4j
        edges = self.get_graph_data()
        edges_df = self.spark.createDataFrame(edges, ["source", "target"])
        
        # Cache in-neighbors
        in_neighbors_dict = self._cache_in_neighbors(edges_df)
        
        all_results = []
        
        for C in decay_factors:
            print(f"\nComputing SimRank with decay factor C = {C}")
            
            results = []
            # Get all unique nodes
            all_nodes = set(row['source'] for row in edges_df.select("source").distinct().collect())
            all_nodes.update(row['target'] for row in edges_df.select("target").distinct().collect())
            
            for query_node in tqdm(query_nodes, desc="Processing query nodes"):
                node_results = []
                for target_node in tqdm(all_nodes, desc=f"Computing similarities for node {query_node}", leave=False):
                    sim = self.compute_simrank_similarity(
                        query_node,
                        target_node,
                        in_neighbors_dict,
                        C
                    )
                    node_results.append((query_node, target_node, sim))
                results.extend(node_results)
            
            results_df = pd.DataFrame(results, columns=['query_node', 'target_node', 'similarity'])
            results_df['decay_factor'] = C
            all_results.append(results_df)
            
            # Save intermediate results
            output_path = f"{output_dir}/simrank_results_C{C}_{timestamp}.csv"
            results_df.to_csv(output_path, index=False)
        
        return self._save_and_summarize_results(all_results, query_nodes, decay_factors, timestamp, output_dir)

    
    def _cache_in_neighbors(self, edges_df):
        """Cache in-neighbors for all nodes"""
        in_neighbors = edges_df.groupBy('target').agg(F.collect_list('source').alias('in_neighbors'))
        return {row['target']: row['in_neighbors'] for row in in_neighbors.collect()}

    
    def _save_and_summarize_results(self, all_results, query_nodes, decay_factors, timestamp, output_dir):
        """Save and summarize final results"""
        final_results = pd.concat(all_results, ignore_index=True)
        
        # Save complete results
        final_path = f"{output_dir}/simrank_all_results_{timestamp}.csv"
        final_results.to_csv(final_path, index=False)
        
        # Generate top results
        top_results = []
        for C in decay_factors:
            for query in query_nodes:
                mask = (final_results['decay_factor'] == C) & (final_results['query_node'] == query)
                subset = final_results[mask].nlargest(10, 'similarity')
                subset = subset.copy()
                subset['rank'] = range(1, len(subset) + 1)
                top_results.append(subset)
        
        top_results_df = pd.concat(top_results, ignore_index=True)
        top_path = f"{output_dir}/simrank_top_results_{timestamp}.csv"
        top_results_df.to_csv(top_path, index=False)
        
        self._print_summary(top_results_df, decay_factors, query_nodes)
        
        return final_results, top_results_df
    
    def _print_summary(self, top_results_df, decay_factors, query_nodes):
        """Print summary of results"""
        print("\nTop 5 most similar nodes for each query node and decay factor:")
        for C in decay_factors:
            print(f"\nDecay factor C = {C}")
            for query in query_nodes:
                print(f"\nQuery node: {query}")
                mask = (top_results_df['decay_factor'] == C) & (top_results_df['query_node'] == query)
                top_5 = top_results_df[mask].head()
                if not top_5.empty:
                    print(top_5[['target_node', 'similarity', 'rank']].to_string())
                else:
                    print("No results found")
    
    def close(self):
        """Close Neo4j and Spark connections"""
        self.driver.close()
        self.spark.stop()

# Example usage
# if __name__ == "__main__":
    # Sample citation data
papers_data = {
    2982615777: [2087551257, 2044328306],
    2044328306: [2087551257],
    2087551257: [2044328306],
    1556418098: []  # Paper with no references
}

# Initialize analyzer
analyzer = CitationGraphAnalyzer()

try:
    
    # Run analysis
    query_nodes = [2982615777, 1556418098]
    decay_factors = [0.7, 0.8, 0.9]
    
    print("Running SimRank analysis...")
    final_results, top_results = analyzer.analyze_citation_graph(
        query_nodes=query_nodes,
        decay_factors=decay_factors
    )
    
finally:
    # Clean up connections
    analyzer.close()

24/11/15 02:28:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Creating Neo4j graph...
Running SimRank analysis...



Computing SimRank with decay factor C = 0.7


Processing query nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Computing similarities for node 2982615777:   0%|          | 0/3 [00:00<?, ?it/s]

...


Computing SimRank with decay factor C = 0.9


Processing query nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Computing similarities for node 2982615777:   0%|          | 0/3 [00:00<?, ?it/s]

Computing similarities for node 1556418098:   0%|          | 0/3 [00:00<?, ?it/s]


Top 5 most similar nodes for each query node and decay factor:

Decay factor C = 0.7

Query node: 2982615777
   target_node  similarity  rank
0   2982615777         1.0     1
1   2044328306         0.0     2
2   2087551257         0.0     3

Query node: 1556418098
   target_node  similarity  rank
3   2982615777         0.0     1
4   2044328306         0.0     2
5   2087551257         0.0     3

Decay factor C = 0.8

Query node: 2982615777
   target_node  similarity  rank
6   2982615777         1.0     1
7   2044328306         0.0     2
8   2087551257         0.0     3

Query node: 1556418098
    target_node  similarity  rank
9    2982615777         0.0     1
10   2044328306         0.0     2
11   2087551257         0.0     3

Decay factor C = 0.9

Query node: 2982615777
    target_node  similarity  rank
12   2982615777         1.0     1
13   2044328306         0.0     2
14   2087551257         0.0     3

Query node: 1556418098
    target_node  similarity  rank
15   2982615777         

In [28]:
from tqdm import tqdm
from neo4j import GraphDatabase
from pyspark.sql import SparkSession
import networkx as nx
import json
from itertools import product
import numpy as np
from typing import List, Dict, Set
import logging
from tqdm.auto import trange

class BatchedCitationGraph:
    def __init__(self, uri: str, user: str, password: str, batch_size: int = 1000):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.batch_size = batch_size
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

    def close(self):
        self.driver.close()

    def clear_database(self):
        """Clear database in batches to avoid memory issues"""
        total_nodes = self.driver.session().run("MATCH (n) RETURN count(n) as count").single()["count"]
        deleted = 0
        with tqdm(total=total_nodes, desc="Clearing database") as pbar:
            while True:
                result = self.driver.session().run(
                    f"MATCH (n) WITH n LIMIT {self.batch_size} "
                    "DETACH DELETE n RETURN count(n)"
                ).single()
                if result[0] == 0:
                    break
                deleted += result[0]
                pbar.update(result[0])

    def create_paper_nodes_batch(self, paper_ids: Set[str]):
        """Create paper nodes in batches"""
        with self.driver.session() as session:
            for i in tqdm(range(0, len(paper_ids), self.batch_size), 
                         desc="Creating paper nodes", 
                         total=(len(paper_ids) + self.batch_size - 1) // self.batch_size):
                batch = list(paper_ids)[i:i + self.batch_size]
                session.run("""
                    UNWIND $batch AS paper_id
                    MERGE (p:Paper {id: paper_id})
                """, batch=batch)

    def create_citation_edges_batch(self, citations: List[tuple]):
        """Create citation edges in batches"""
        with self.driver.session() as session:
            for i in tqdm(range(0, len(citations), self.batch_size), 
                         desc="Creating citation edges",
                         total=(len(citations) + self.batch_size - 1) // self.batch_size):
                batch = citations[i:i + self.batch_size]
                session.run("""
                    UNWIND $batch AS citation
                    MATCH (p1:Paper {id: citation[0]})
                    MATCH (p2:Paper {id: citation[1]})
                    MERGE (p1)-[:CITES]->(p2)
                """, batch=batch)

def create_neo4j_graph(data_file: str, batch_size: int = 1000):
    """Create Neo4j graph with batched operations"""
    graph = BatchedCitationGraph(
        "bolt://localhost:7687",
        "neo4j",
        "paras2003",
        batch_size=batch_size
    )

    # Clear existing data
    graph.logger.info("Starting database cleanup...")
    graph.clear_database()

    # First pass: Collect all unique paper IDs
    paper_ids = set()
    citations = []
    
    # Count total lines in file first
    total_lines = sum(1 for _ in open(data_file, 'r'))
    
    graph.logger.info("Processing data file...")
    with open(data_file, 'r') as f:
        with tqdm(f, total=total_lines, desc="Reading papers") as pbar:
            for line in pbar:
                record = json.loads(line)
                citing_paper = record["paper"]
                references = record["reference"]
                
                paper_ids.add(citing_paper)
                paper_ids.update(references)
                citations.extend((citing_paper, cited_paper) 
                               for cited_paper in references)
                pbar.set_postfix({'papers': len(paper_ids), 'citations': len(citations)})

    # Create nodes in batches
    graph.logger.info(f"Creating {len(paper_ids)} paper nodes...")
    graph.create_paper_nodes_batch(paper_ids)

    # Create edges in batches
    graph.logger.info(f"Creating {len(citations)} citation edges...")
    graph.create_citation_edges_batch(citations)

    graph.logger.info("Graph creation completed")
    graph.close()

def convert_to_networkx(neo4j_driver, batch_size: int = 1000):
    """Convert Neo4j graph to NetworkX graph in batches"""
    G = nx.DiGraph()
    
    with neo4j_driver.session() as session:
        # Get total counts for progress bars
        total_nodes = session.run("MATCH (p:Paper) RETURN count(p) as count").single()["count"]
        total_edges = session.run("MATCH ()-[:CITES]->() RETURN count(*) as count").single()["count"]
        
        # Process nodes in batches
        with tqdm(total=total_nodes, desc="Adding nodes to NetworkX") as pbar:
            for offset in range(0, total_nodes, batch_size):
                nodes = session.run(
                    f"MATCH (p:Paper) RETURN p.id AS id "
                    f"SKIP {offset} LIMIT {batch_size}"
                )
                batch_nodes = [node["id"] for node in nodes]
                G.add_nodes_from(batch_nodes)
                pbar.update(len(batch_nodes))
        
        # Process edges in batches
        with tqdm(total=total_edges, desc="Adding edges to NetworkX") as pbar:
            processed = 0
            while processed < total_edges:
                edges = session.run(f"""
                    MATCH (p1:Paper)-[:CITES]->(p2:Paper) 
                    RETURN p1.id AS source, p2.id AS target 
                    SKIP {processed} LIMIT {batch_size}
                """).data()
                
                if not edges:
                    break
                
                G.add_edges_from((edge["source"], edge["target"]) for edge in edges)
                processed += len(edges)
                pbar.update(len(edges))
            
    return G

def simrank_spark(G, query_nodes, C, max_iterations=100, tolerance=0.0001, partition_size=1000):
    """Memory-optimized SimRank implementation with progress tracking"""
    spark = SparkSession.builder \
        .appName("SimRank") \
        .config("spark.driver.memory", "4g") \
        .config("spark.executor.memory", "4g") \
        .config("spark.memory.offHeap.enabled", "true") \
        .config("spark.memory.offHeap.size", "4g") \
        .config("spark.driver.bindAddress", "127.0.0.1") \
        .getOrCreate()


    # Process predecessors in batches
    predecessors = {}
    nodes = list(G.nodes())
    with tqdm(total=len(nodes), desc="Building predecessor map") as pbar:
        for i in range(0, len(nodes), partition_size):
            batch_nodes = nodes[i:i + partition_size]
            for node in batch_nodes:
                predecessors[node] = list(G.predecessors(node))
            pbar.update(len(batch_nodes))

    node_to_idx = {node: i for i, node in enumerate(nodes)}
    n = len(nodes)

    # Initialize similarity matrix using sparse representation
    from scipy.sparse import lil_matrix
    sim_matrix = lil_matrix((n, n))
    for i in tqdm(range(n), desc="Initializing similarity matrix"):
        sim_matrix[i, i] = 1.0

    sim_rdd = spark.sparkContext.broadcast(sim_matrix.tocsr())

    def compute_pair_similarity(u, v):
        if u == v:
            return 1.0
        
        in_u = predecessors[u]
        in_v = predecessors[v]
        
        if not in_u or not in_v:
            return 0.0
        
        scale = C / (len(in_u) * len(in_v))
        curr_sim = sim_rdd.value
        
        sum_sim = 0
        for w, x in product(in_u, in_v):
            sum_sim += curr_sim[node_to_idx[w], node_to_idx[x]]
        
        return scale * sum_sim

    # Process iterations
    pbar = tqdm(total=max_iterations, desc="SimRank iterations")
    for iteration in range(max_iterations):
        new_sim = lil_matrix((n, n))
        
        # Process node pairs in batches with nested progress bars
        total_batches = ((n + partition_size - 1) // partition_size) ** 2
        batch_pbar = tqdm(total=total_batches, desc=f"Processing batch pairs", leave=False)
        
        for i in range(0, n, partition_size):
            for j in range(0, n, partition_size):
                batch_pairs = [(nodes[ii], nodes[jj]) 
                             for ii in range(i, min(i + partition_size, n))
                             for jj in range(j, min(j + partition_size, n))]
                
                node_pairs_rdd = spark.sparkContext.parallelize(batch_pairs)
                similarities = node_pairs_rdd.map(lambda pair: 
                    (pair, compute_pair_similarity(pair[0], pair[1]))).collect()
                
                for (u, v), sim in similarities:
                    new_sim[node_to_idx[u], node_to_idx[v]] = sim
                
                batch_pbar.update(1)
        
        batch_pbar.close()
        
        # Check convergence
        diff = abs(new_sim - sim_rdd.value).max()
        sim_rdd = spark.sparkContext.broadcast(new_sim.tocsr())
        
        pbar.set_postfix({'diff': f'{diff:.6f}'})
        pbar.update(1)
        
        if diff < tolerance:
            logging.info(f"Converged after {iteration + 1} iterations")
            break
    
    pbar.close()
    
    # Get results for query nodes
    results = {}
    for query_node in tqdm(query_nodes, desc="Processing query nodes"):
        if query_node not in node_to_idx:
            logging.warning(f"Query node {query_node} not found in graph")
            continue
            
        query_idx = node_to_idx[query_node]
        similarities = []
        
        # Process similarities in batches
        for i in range(0, n, partition_size):
            batch_nodes = nodes[i:i + partition_size]
            batch_similarities = [(node, sim_rdd.value[query_idx, node_to_idx[node]])
                                for node in batch_nodes if node != query_node]
            similarities.extend(batch_similarities)
        
        similarities.sort(key=lambda x: x[1], reverse=True)
        results[query_node] = similarities
    
    spark.stop()
    return results
def main():
    # Create Neo4j graph with smaller batch size
    create_neo4j_graph("train.json", batch_size=500)
    
    neo4j_connection = GraphDatabase.driver(
        "bolt://localhost:7687",
        auth=("neo4j", "paras2003")
    )
    
    # Convert to NetworkX with batching
    G = convert_to_networkx(neo4j_connection, batch_size=500)
    neo4j_connection.close()
    
    query_nodes = ["2982615777", "1556418098"]
    results = simrank_spark(G, query_nodes, C=0.8)


In [29]:
from networkx.algorithms.similarity import simrank_similarity
def create_neo4j_graph(data_file: str, batch_size: int = 1000):
    """Create Neo4j graph with batched operations"""
    graph = BatchedCitationGraph(
        "bolt://localhost:7687",
        "neo4j",
        "paras2003",
        batch_size=batch_size
    )

    # Clear existing data
    graph.logger.info("Starting database cleanup...")
    graph.clear_database()

    # First pass: Collect all unique paper IDs
    paper_ids = set()
    citations = []
    
    # Count total lines in file first
    total_lines = sum(1 for _ in open(data_file, 'r'))
    
    graph.logger.info("Processing data file...")
    with open(data_file, 'r') as f:
        with tqdm(f, total=total_lines, desc="Reading papers") as pbar:
            for line in pbar:
                record = json.loads(line)
                citing_paper = record["paper"]
                references = record["reference"]
                
                paper_ids.add(citing_paper)
                paper_ids.update(references)
                citations.extend((citing_paper, cited_paper) 
                               for cited_paper in references)
                pbar.set_postfix({'papers': len(paper_ids), 'citations': len(citations)})

    # Create nodes in batches
    graph.logger.info(f"Creating {len(paper_ids)} paper nodes...")
    graph.create_paper_nodes_batch(paper_ids)

    # Create edges in batches
    graph.logger.info(f"Creating {len(citations)} citation edges...")
    graph.create_citation_edges_batch(citations)

    graph.logger.info("Graph creation completed")
    graph.close()
def compute_simrank(G, query_nodes, decay_factor=0.8):
    """
    Compute SimRank using NetworkX's built-in function.
    
    Args:
        G (networkx.DiGraph): The citation graph.
        query_nodes (List[str]): Nodes for which to compute SimRank.
        decay_factor (float): The decay factor for SimRank computation.
    
    Returns:
        Dict[str, List[Tuple[str, float]]]: Similarity scores for query nodes.
    """
    # Compute SimRank for the entire graph
    simrank_scores = simrank_similarity(G, decay_factor=decay_factor)

    # Extract and filter results for query nodes
    results = {}
    for query_node in query_nodes:
        if query_node not in G.nodes:
            logging.warning(f"Query node {query_node} not found in graph")
            continue
        
        # Sort similarities for the query node, excluding self-similarity
        similarities = sorted(
            ((target_node, sim) for target_node, sim in simrank_scores[query_node].items() if target_node != query_node),
            key=lambda x: x[1],
            reverse=True
        )
        results[query_node] = similarities
    
    return results


In [30]:

# Create Neo4j graph with smaller batch size
create_neo4j_graph("train.json", batch_size=500)

neo4j_connection = GraphDatabase.driver(
    "bolt://localhost:7687",
    auth=("neo4j", "paras2003")
)

# Convert to NetworkX with batching
G = convert_to_networkx(neo4j_connection, batch_size=500)
neo4j_connection.close()

# Query nodes for SimRank
query_nodes = ["2982615777", "1556418098"]
decay_factor = 0.8

# Compute SimRank using NetworkX
results = compute_simrank(G, query_nodes, decay_factor=decay_factor)

# Output results
for query_node, similarities in results.items():
    print(f"SimRank results for query node {query_node}:")
    for target_node, similarity in similarities[:10]:  # Top 10 most similar nodes
        print(f"  {target_node}: {similarity:.4f}")


Adding edges to NetworkX: 100%|██████████| 1216784/1216784 [09:56<00:00, 2040.51it/s]


TypeError: simrank_similarity() got an unexpected keyword argument 'decay_factor'

In [39]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from neo4j import GraphDatabase
import pandas as pd
import os
from datetime import datetime
from tqdm.auto import tqdm

class CitationGraphAnalyzer:
    def __init__(self, neo4j_uri="bolt://localhost:7687", 
                 neo4j_user="neo4j", neo4j_password="paras2003"):
        """Initialize with Neo4j and Spark connections"""
        self.driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))
        self.spark = SparkSession.builder \
            .appName("Citation Graph Analysis") \
            .config("spark.driver.memory", "4g") \
            .config("spark.executor.memory", "4g") \
            .config("spark.task.maxFailures", "4") \
            .getOrCreate()

    def get_graph_data(self):
        """Extract graph data from Neo4j for Spark processing"""
        with self.driver.session() as session:
            result = session.run("""
                MATCH (p1:Paper)-[:CITES]->(p2:Paper)
                RETURN p1.id as source, p2.id as target
            """)
            edges = [(record["source"], record["target"]) for record in result]
            return edges

    def compute_simrank(self, edges_df, query_nodes, decay_factors, output_dir="simrank_results", max_iterations=10, tolerance=1e-4):
        """Run SimRank analysis on citation graph"""
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Ensure column names are standardized
        edges_df = edges_df.withColumnRenamed("src", "source").withColumnRenamed("dst", "target")
        
        # Cache in-neighbors
        in_neighbors_dict = self._cache_in_neighbors(edges_df)
        
        all_results = []
        
        for C in decay_factors:
            print(f"\nComputing SimRank with decay factor C = {C}")
            
            results = []
            all_nodes = set(row['source'] for row in edges_df.select("source").distinct().collect())
            all_nodes.update(row['target'] for row in edges_df.select("target").distinct().collect())
            
            for query_node in tqdm(query_nodes, desc="Processing query nodes"):
                node_results = []
                for target_node in tqdm(all_nodes, desc=f"Computing similarities for node {query_node}", leave=False):
                    sim = self._compute_simrank_similarity(
                        query_node,
                        target_node,
                        in_neighbors_dict,
                        C,
                        max_iterations,
                        tolerance
                    )
                    node_results.append((query_node, target_node, sim))
                results.extend(node_results)
            
            results_df = pd.DataFrame(results, columns=['query_node', 'target_node', 'similarity'])
            results_df['decay_factor'] = C
            all_results.append(results_df)
            
            # Save intermediate results
            output_path = f"{output_dir}/simrank_results_C{C}_{timestamp}.csv"
            results_df.to_csv(output_path, index=False)
        
        return self._save_and_summarize_results(all_results, query_nodes, decay_factors, timestamp, output_dir)

    def _cache_in_neighbors(self, edges_df):
        """Cache in-neighbors for all nodes"""
        in_neighbors = edges_df.groupBy('target').agg(F.collect_list('source').alias('in_neighbors'))
        return {row['target']: row['in_neighbors'] for row in in_neighbors.collect()}

    def _compute_simrank_similarity(self, a, b, in_neighbors_dict, C, max_iterations, tolerance):
        """Compute SimRank similarity between two nodes."""
        if a == b:
            return 1.0
            
        in_neighbors_a = in_neighbors_dict.get(a, [])
        in_neighbors_b = in_neighbors_dict.get(b, [])
        
        if not in_neighbors_a or not in_neighbors_b:
            return 0.0
        
        # Initialize similarity matrix for in-neighbors
        sim_matrix = {}
        for na in in_neighbors_a:
            for nb in in_neighbors_b:
                sim_matrix[(na, nb)] = 0.0
        
        # Iterate until convergence
        for _ in range(max_iterations):
            new_sim_matrix = {}
            max_diff = 0.0
            
            for na in in_neighbors_a:
                for nb in in_neighbors_b:
                    in_na = in_neighbors_dict.get(na, [])
                    in_nb = in_neighbors_dict.get(nb, [])
                    
                    if not in_na or not in_nb:
                        continue
                    
                    sum_sim = sum(sim_matrix.get((i, j), 0.0) for i in in_na for j in in_nb)
                    new_sim = (C / (len(in_na) * len(in_nb))) * sum_sim
                    new_sim_matrix[(na, nb)] = new_sim
                    max_diff = max(max_diff, abs(new_sim - sim_matrix.get((na, nb), 0.0)))
            
            sim_matrix = new_sim_matrix
            if max_diff < tolerance:
                break
        
        sum_sim = sum(sim_matrix.get((na, nb), 0.0) for na in in_neighbors_a for nb in in_neighbors_b)
        return (C / (len(in_neighbors_a) * len(in_neighbors_b))) * sum_sim

    def _save_and_summarize_results(self, all_results, query_nodes, decay_factors, timestamp, output_dir):
        """Save and summarize final results"""
        final_results = pd.concat(all_results, ignore_index=True)
        final_results_path = f"{output_dir}/final_simrank_results_{timestamp}.csv"
        final_results.to_csv(final_results_path, index=False)
        
        top_results = final_results.groupby(['query_node', 'decay_factor']).apply(
            lambda group: group.nlargest(10, 'similarity')).reset_index(drop=True)
        top_results_path = f"{output_dir}/top_simrank_results_{timestamp}.csv"
        top_results.to_csv(top_results_path, index=False)
        
        return final_results, top_results

    def close(self):
        """Close Neo4j and Spark connections"""
        self.driver.close()
        self.spark.stop()

# Example usage
if __name__ == "__main__":
    edges_csv_path = "edges.csv"
    edges_df = pd.read_csv(edges_csv_path)
    
    spark = SparkSession.builder.appName("SimRank Analysis").getOrCreate()
    edges_sdf = spark.createDataFrame(edges_df)

    analyzer = CitationGraphAnalyzer()

    try:
        query_nodes = [2982615777, 1556418098]
        decay_factors = [0.7, 0.8, 0.9]
        print("Running SimRank analysis...")
        final_results, top_results = analyzer.compute_simrank(
            edges_sdf,
            query_nodes=query_nodes,
            decay_factors=decay_factors
        )
    finally:
        analyzer.close()


24/11/15 19:14:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Running SimRank analysis...


24/11/15 19:15:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
24/11/15 19:15:19 WARN TaskSetManager: Stage 0 contains a task of very large size (2001 KiB). The maximum recommended task size is 1000 KiB.



Computing SimRank with decay factor C = 0.7


24/11/15 19:15:24 WARN TaskSetManager: Stage 3 contains a task of very large size (2001 KiB). The maximum recommended task size is 1000 KiB.
24/11/15 19:15:25 WARN TaskSetManager: Stage 6 contains a task of very large size (2001 KiB). The maximum recommended task size is 1000 KiB.


...


Computing SimRank with decay factor C = 0.9


24/11/15 19:17:08 WARN TaskSetManager: Stage 15 contains a task of very large size (2001 KiB). The maximum recommended task size is 1000 KiB.
24/11/15 19:17:10 WARN TaskSetManager: Stage 18 contains a task of very large size (2001 KiB). The maximum recommended task size is 1000 KiB.


Processing query nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Computing similarities for node 2982615777:   0%|          | 0/359143 [00:00<?, ?it/s]

Computing similarities for node 1556418098:   0%|          | 0/359143 [00:00<?, ?it/s]

In [48]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from neo4j import GraphDatabase
import pandas as pd
import os
from datetime import datetime
from tqdm.auto import tqdm

class CitationGraphAnalyzer:
    def __init__(self, neo4j_uri="bolt://localhost:7687", 
                 neo4j_user="neo4j", neo4j_password="paras2003"):
        """Initialize with Neo4j and Spark connections"""
        self.driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))
        self.spark = SparkSession.builder \
            .appName("Citation Graph Analysis") \
            .config("spark.driver.memory", "4g") \
            .config("spark.executor.memory", "4g") \
            .config("spark.task.maxFailures", "4") \
            .getOrCreate()

    def get_graph_data(self):
        """Extract graph data from Neo4j for Spark processing"""
        with self.driver.session() as session:
            result = session.run("""
                MATCH (p1:Paper)-[:CITES]->(p2:Paper)
                RETURN p1.id as source, p2.id as target
            """)
            edges = [(record["source"], record["target"]) for record in result]
            return edges

    def compute_simrank(self, edges_df, query_nodes, decay_factors, output_dir="simrank_results", max_iterations=10, tolerance=1e-4):
        """Run SimRank analysis on citation graph"""
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Ensure column names are standardized
        edges_df = edges_df.withColumnRenamed("src", "source").withColumnRenamed("dst", "target")
        
        # Cache in-neighbors
        in_neighbors_dict = self._cache_in_neighbors(edges_df)
        
        all_results = []
        
        for C in decay_factors:
            print(f"\nComputing SimRank with decay factor C = {C}")
            
            results = []
            all_nodes = set(row['source'] for row in edges_df.select("source").distinct().collect())
            all_nodes.update(row['target'] for row in edges_df.select("target").distinct().collect())
            
            for query_node in tqdm(query_nodes, desc="Processing query nodes"):
                node_results = []
                for target_node in tqdm(all_nodes, desc=f"Computing similarities for node {query_node}", leave=False):
                    sim = self._compute_simrank_similarity(
                        query_node,
                        target_node,
                        in_neighbors_dict,
                        C,
                        max_iterations,
                        tolerance
                    )
                    node_results.append((query_node, target_node, sim))
                results.extend(node_results)
            
            results_df = pd.DataFrame(results, columns=['query_node', 'target_node', 'similarity'])
            results_df['decay_factor'] = C
            all_results.append(results_df)
            
            # Save intermediate results
            output_path = f"{output_dir}/simrank_results_C{C}_{timestamp}.csv"
            results_df.to_csv(output_path, index=False)
        
        return self._save_and_summarize_results(all_results, query_nodes, decay_factors, timestamp, output_dir)

    def _cache_in_neighbors(self, edges_df):
        """Cache in-neighbors for all nodes"""
        in_neighbors = edges_df.groupBy('target').agg(F.collect_list('source').alias('in_neighbors'))
        return {row['target']: row['in_neighbors'] for row in in_neighbors.collect()}

    def _compute_simrank_similarity(self, a, b, in_neighbors_dict, C, max_iterations, tolerance):
        """Compute SimRank similarity between two nodes."""
        if a == b:
            return 1.0
        
        in_neighbors_a = in_neighbors_dict.get(a, [])
        in_neighbors_b = in_neighbors_dict.get(b, [])
        
        if not in_neighbors_a or not in_neighbors_b:
            return 0.0
        
        # Initialize similarity matrix for in-neighbors
        sim_matrix = {}
        for na in in_neighbors_a:
            for nb in in_neighbors_b:
                sim_matrix[(na, nb)] = 0.0
        
        # Iterate until convergence
        for _ in range(max_iterations):
            new_sim_matrix = {}
            max_diff = 0.0
            
            for na in in_neighbors_a:
                for nb in in_neighbors_b:
                    in_na = in_neighbors_dict.get(na, [])
                    in_nb = in_neighbors_dict.get(nb, [])
                    
                    if not in_na or not in_nb:
                        continue
                    
                    sum_sim = sum(sim_matrix.get((i, j), 0.0) for i in in_na for j in in_nb)
                    new_sim = (C / (len(in_na) * len(in_nb))) * sum_sim
                    new_sim_matrix[(na, nb)] = new_sim
                    max_diff = max(max_diff, abs(new_sim - sim_matrix.get((na, nb), 0.0)))
            
            sim_matrix = new_sim_matrix
            if max_diff < tolerance:
                break
        
        sum_sim = sum(sim_matrix.get((na, nb), 0.0) for na in in_neighbors_a for nb in in_neighbors_b)
        normalization_factor = (len(in_neighbors_a) * len(in_neighbors_b)) or 1  # Prevent division by 0
        return (C / normalization_factor) * sum_sim

    def _save_and_summarize_results(self, all_results, query_nodes, decay_factors, timestamp, output_dir):
        """Save and summarize final results"""
        final_results = pd.concat(all_results, ignore_index=True)
        final_results_path = f"{output_dir}/final_simrank_results_{timestamp}.csv"
        final_results.to_csv(final_results_path, index=False)
        
        top_results = final_results.groupby(['query_node', 'decay_factor']).apply(
            lambda group: group.nlargest(10, 'similarity')).reset_index(drop=True)
        top_results_path = f"{output_dir}/top_simrank_results_{timestamp}.csv"
        top_results.to_csv(top_results_path, index=False)
        
        return final_results, top_results

    def close(self):
        """Close Neo4j and Spark connections"""
        self.driver.close()
        self.spark.stop()

    def bfs_traversal(self, start_node, depth=2):
        """Perform BFS to get all nodes within a given depth"""
        with self.driver.session() as session:
            result = session.run("""
                MATCH (start:Paper)-[:CITES*1..{depth}]->(p:Paper)
                WHERE start.id = $start_node
                RETURN p.id as node
            """, start_node=start_node, depth=depth)
            return [record["node"] for record in result]

# Example usage
if __name__ == "__main__":
    edges_csv_path = "graph_edges.csv"
    edges_df = pd.read_csv(edges_csv_path)
    
    spark = SparkSession.builder.appName("SimRank Analysis").getOrCreate()
    edges_sdf = spark.createDataFrame(edges_df)

    analyzer = CitationGraphAnalyzer()

    try:
        query_nodes = [2982615777, 1556418098]
        decay_factors = [0.7, 0.8, 0.9]
        print("Running SimRank analysis...")
        final_results, top_results = analyzer.compute_simrank(
            edges_sdf,
            query_nodes=query_nodes,
            decay_factors=decay_factors
        )
    finally:
        analyzer.close()


24/11/15 19:37:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/15 19:37:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
24/11/15 19:37:21 WARN TaskSetManager: Stage 0 contains a task of very large size (2001 KiB). The maximum recommended task size is 1000 KiB.


Running SimRank analysis...



Computing SimRank with decay factor C = 0.7


24/11/15 19:37:24 WARN TaskSetManager: Stage 3 contains a task of very large size (2001 KiB). The maximum recommended task size is 1000 KiB.
24/11/15 19:37:26 WARN TaskSetManager: Stage 6 contains a task of very large size (2001 KiB). The maximum recommended task size is 1000 KiB.


Processing query nodes:   0%|          | 0/2 [00:00<?, ?it/s]

...


Computing SimRank with decay factor C = 0.9


24/11/15 19:39:08 WARN TaskSetManager: Stage 15 contains a task of very large size (2001 KiB). The maximum recommended task size is 1000 KiB.
24/11/15 19:39:10 WARN TaskSetManager: Stage 18 contains a task of very large size (2001 KiB). The maximum recommended task size is 1000 KiB.


Processing query nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Computing similarities for node 2982615777:   0%|          | 0/359143 [00:00<?, ?it/s]

Computing similarities for node 1556418098:   0%|          | 0/359143 [00:00<?, ?it/s]